In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!ls /content/drive/MyDrive

 0_0_addtochaos_0-20250416T234111Z-001.zip  'Colab Notebooks'   Image_Vision


In [24]:
import zipfile
import os

zip_path = "/mnt/gdrive/MyDrive/Image_Vision/24932499.zip"
extract_to = "/mnt/gdrive/MyDrive/Image_Vision/test_set"

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("✅ Successfully extracted test set!")

✅ Successfully extracted test set!


In [15]:
# Unzip test videos
import os
import zipfile

zip_dir = "/mnt/gdrive/MyDrive/Image_Vision/test_set"
output_dir = "/mnt/gdrive/MyDrive/Image_Vision/test_videos"

os.makedirs(output_dir, exist_ok=True)

for file in sorted(os.listdir(zip_dir)):
    if file.endswith(".zip") and file.startswith("video_"):
        zip_path = os.path.join(zip_dir, file)
        folder_name = file.replace(".zip", "")
        extract_path = os.path.join(output_dir, folder_name)

        if not os.path.exists(extract_path):
            print(f"🔓 Unzipping {file}...")
            try:
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_path)
                print(f"✅ Done: {folder_name}")
            except zipfile.BadZipFile:
                print(f"❌ Skipping {file} — not a valid zip.")
        else:
            print(f"🟡 Already unzipped: {folder_name}")


🔓 Unzipping video_41.zip...
✅ Done: video_41
🔓 Unzipping video_42.zip...
✅ Done: video_42
🔓 Unzipping video_43.zip...
✅ Done: video_43
🔓 Unzipping video_44.zip...
✅ Done: video_44
🔓 Unzipping video_45.zip...
✅ Done: video_45
🔓 Unzipping video_46.zip...
✅ Done: video_46
🔓 Unzipping video_47.zip...
✅ Done: video_47
🔓 Unzipping video_48.zip...
✅ Done: video_48
🔓 Unzipping video_49.zip...
✅ Done: video_49
🔓 Unzipping video_50.zip...
✅ Done: video_50


In [16]:
# Check whats inside
!ls -lh /mnt/gdrive/MyDrive/Image_Vision/test_videos/video_41

total 473M
-rw------- 1 root root  406 Apr 17 17:18 action_continuous.txt
-rw------- 1 root root  28K Apr 17 17:18 action_discrete.txt
drwx------ 2 root root 4.0K Apr 17 17:18 segmentation
-rw------- 1 root root 473M Apr 17 17:18 video_left.avi


In [17]:
#Extract Frames from All Test Videos
import cv2
import os

def extract_frames(video_path, output_dir, every_n=5):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_idx = 0
    saved_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % every_n == 0:
            out_path = os.path.join(output_dir, f"{frame_idx:09d}.jpg")
            cv2.imwrite(out_path, frame)
            saved_idx += 1
        frame_idx += 1

    cap.release()
    print(f"✅ Extracted {saved_idx} frames from {os.path.basename(video_path)}")

# 🔁 Batch extract frames from all test videos
video_base = "/mnt/gdrive/MyDrive/Image_Vision/test_videos"
output_base = "/mnt/gdrive/MyDrive/Image_Vision/test_frames"

for folder in sorted(os.listdir(video_base)):
    video_path = os.path.join(video_base, folder, "video_left.avi")
    output_dir = os.path.join(output_base, folder)

    if os.path.exists(video_path):
        print(f"🎞 Extracting frames from: {folder}")
        extract_frames(video_path, output_dir, every_n=5)
    else:
        print(f"⚠️ Missing video in {folder}, skipping...")


🎞 Extracting frames from: video_41
✅ Extracted 2856 frames from video_left.avi
🎞 Extracting frames from: video_42
✅ Extracted 4289 frames from video_left.avi
🎞 Extracting frames from: video_43
✅ Extracted 2371 frames from video_left.avi
🎞 Extracting frames from: video_44
✅ Extracted 1715 frames from video_left.avi
🎞 Extracting frames from: video_45
✅ Extracted 1395 frames from video_left.avi
🎞 Extracting frames from: video_46
✅ Extracted 7586 frames from video_left.avi
🎞 Extracting frames from: video_47
✅ Extracted 3999 frames from video_left.avi
🎞 Extracting frames from: video_48
✅ Extracted 3602 frames from video_left.avi
🎞 Extracting frames from: video_49
✅ Extracted 8237 frames from video_left.avi
🎞 Extracting frames from: video_50
✅ Extracted 2916 frames from video_left.avi


In [10]:
#Load Trained Model

import torch

import torch.nn as nn

class UNet(nn.Module):
    def __init__(self, n_classes):
        super(UNet, self).__init__()

        def conv_block(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels, out_channels, 3, padding=1),
                nn.ReLU(inplace=True),
            )

        self.down1 = conv_block(3, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.down2 = conv_block(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.down3 = conv_block(128, 256)
        self.pool3 = nn.MaxPool2d(2)

        self.middle = conv_block(256, 512)

        self.up3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.upconv3 = conv_block(512, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.upconv2 = conv_block(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.upconv1 = conv_block(128, 64)

        self.final = nn.Conv2d(64, n_classes, kernel_size=1)

    def forward(self, x):
        d1 = self.down1(x)
        p1 = self.pool1(d1)
        d2 = self.down2(p1)
        p2 = self.pool2(d2)
        d3 = self.down3(p2)
        p3 = self.pool3(d3)

        m = self.middle(p3)

        u3 = self.up3(m)
        u3 = self.upconv3(torch.cat([u3, d3], dim=1))
        u2 = self.up2(u3)
        u2 = self.upconv2(torch.cat([u2, d2], dim=1))
        u1 = self.up1(u2)
        u1 = self.upconv1(torch.cat([u1, d1], dim=1))

        return self.final(u1)



# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("⚡ Using device:", device)


model_path = "/mnt/gdrive/MyDrive/Image_Vision/models/best_unet.pth"

model = UNet(n_classes=10).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/Image_Vision/models/best_unet.pth"))
model.eval()
print("✅ Model loaded!")

⚡ Using device: cuda
✅ Model loaded!


In [ ]:
import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
from torchvision import transforms

from PIL import Image

# --- Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(n_classes=10).to(device)  # 🔁 make sure UNet is already defined
model.load_state_dict(torch.load("/content/drive/MyDrive/Image_Vision/models/best_unet.pth"))
model.eval()
print("✅ Model loaded!")

# --- Paths ---
test_videos = [f"video_{i}" for i in range(41, 51)]
test_frames_root = "/content/drive/MyDrive/Image_Vision/test_frames"
output_root = "/content/drive/MyDrive/Image_Vision/test_predictions"
rgb_preview_root = "/content/drive/MyDrive/Image_Vision/test_predictions_rgb"  # optional

# --- Label to color map for preview visualization ---
label_to_color = {
    0: (0, 0, 0),         # background
    1: (255, 0, 0),
    2: (0, 255, 0),
    3: (0, 0, 255),
    4: (255, 255, 0),
    5: (0, 255, 255),
    6: (128, 0, 255),
    7: (255, 0, 255),
    8: (0, 128, 255),
    9: (255, 255, 255)
}

# --- Frame prediction ---
def predict_frame(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (384, 384))
    tensor = transforms.ToTensor()(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(tensor)
        pred = torch.argmax(output.squeeze(), dim=0).cpu().numpy().astype(np.uint8)

    return pred  # raw class mask (0-9)

# --- Optional: Convert class mask to RGB for preview ---
def mask_to_rgb(mask):
    rgb = np.zeros((384, 384, 3), dtype=np.uint8)
    for label, color in label_to_color.items():
        rgb[mask == label] = color
    return rgb

# --- Run predictions ---
for video in test_videos:
    print(f"🎯 Predicting for {video}")
    input_dir = os.path.join(test_frames_root, video)
    output_dir = os.path.join(output_root, video)
    preview_dir = os.path.join(rgb_preview_root, video)
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(preview_dir, exist_ok=True)

    frame_list = sorted([f for f in os.listdir(input_dir) if f.endswith(".jpg")])

    for frame_name in tqdm(frame_list):
        img_path = os.path.join(input_dir, frame_name)
        pred_mask = predict_frame(img_path)

        # Save raw class mask (for evaluation)
        out_path = os.path.join(output_dir, frame_name.replace(".jpg", ".png"))
        cv2.imwrite(out_path, pred_mask)

        # Save RGB mask (for preview)
        rgb_mask = mask_to_rgb(pred_mask)
        preview_path = os.path.join(preview_dir, frame_name.replace(".jpg", ".png"))
        Image.fromarray(rgb_mask).save(preview_path)


✅ Model loaded!
🎯 Predicting for video_41


100%|██████████| 2856/2856 [02:10<00:00, 21.92it/s]


🎯 Predicting for video_42


 99%|█████████▉| 4261/4289 [18:17<00:16,  1.68it/s]